In [1]:
import os 
import sys 
sys.path.append("../training")
import pickle

import hashlib
import hls4ml 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.metrics import accuracy_score

import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Activation, BatchNormalization, Dropout, Softmax
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.losses import CategoricalCrossentropy

from tensorflow_model_optimization.python.core.sparsity.keras import prune, pruning_callbacks, pruning_schedule
from tensorflow_model_optimization.sparsity.keras import strip_pruning
import tensorflow_model_optimization as tfmot

from qkeras.qlayers import QDense, QActivation
from qkeras import QBatchNormalization
from qkeras.quantizers import quantized_bits, quantized_relu
from qkeras.utils import _add_supported_quantized_objects
from tensorflow.keras.models import load_model
from qkeras.utils import _add_supported_quantized_objects

2024-08-20 12:34:40.194480: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-08-20 12:34:40.294617: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-08-20 12:34:40.294636: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-08-20 12:34:40.797065: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directo

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'


/home/jcampos/miniforge3/envs/ml4qick-env/lib/python3.8/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


## Load Data

#### Impoartant! 
Download the dataset locally from [OneDrive here](https://purdue0-my.sharepoint.com/personal/du245_purdue_edu/_layouts/15/onedrive.aspx?id=%2Fpersonal%2Fdu245%5Fpurdue%5Fedu%2FDocuments%2FShared%2FQSC%20ML%20for%20readout%2FFinal%5Fraw%5Fdata%5Ffor%5Fpaper%2Fdata%5F0528%5Fnpy). We are using QICK data with timestamp **0528**. 

In [2]:
def one_hot_encode(data):
    y_encoded = np.zeros([data.shape[0],2], dtype=np.int32)
    for idx, x in enumerate(data):
        if x == 1:
            y_encoded[idx][1] = 1
        else:
            y_encoded[idx][0] = 1
    return y_encoded

In [3]:
"""Loadning training split"""
start_window = 0
end_window = 770
data_dir = "../data/malab_05282024/npz/"
assert os.path.exists(f"{data_dir}/0528_X_train_{start_window}_{end_window}.npy"), "File does not exist "

x_train_path = os.path.join(data_dir, f'0528_X_train_{start_window}_{end_window}.npy')
y_train_path = os.path.join(data_dir, f'0528_y_train_{start_window}_{end_window}.npy')

X_train_val = np.load(x_train_path)
y_train_val = np.load(y_train_path)

# Insure same dataset is loaded 
assert hashlib.md5(X_train_val).hexdigest() == 'b61226c86b7dee0201a9158455e08ffb',  "Checksum failed. Wrong file was loaded or file may be corrupted."
assert hashlib.md5(y_train_val).hexdigest() == 'c59ce37dc7c73d2d546e7ea180fa8d31',  "Checksum failed. Wrong file was loaded or file may be corrupted."

y_train_val = one_hot_encode(y_train_val)

print("Train Data Set:")
print("\tX Path        :", x_train_path)
print("\ty Path        :", y_train_path)
print("\tSize          :", len(X_train_val))
print("\tSample Shape  :", X_train_val[0].shape)
print("\tMean          :", X_train_val.mean())
print("\tStd. Dev.     :", X_train_val.std())

assert len(X_train_val[0]) == (end_window-start_window)*2, "ERROR: Specified window does not match loaded dataset shape"

Train Data Set:
	X Path        : ../data/malab_05282024/npz/0528_X_train_0_770.npy
	y Path        : ../data/malab_05282024/npz/0528_y_train_0_770.npy
	Size          : 900000
	Sample Shape  : (1540,)
	Mean          : 57.37779754545455
	Std. Dev.     : 844.0956096913322


In [4]:
"""Loading testing split"""
start_window = 0
end_window = 770
data_dir = "../data/malab_05282024/npz/"
assert os.path.exists(f"{data_dir}/X_test_{start_window}_{end_window}.npy"), "File does not exist "

x_test_path = os.path.join(data_dir, f'0528_X_test_{start_window}_{end_window}.npy')
y_test_path = os.path.join(data_dir, f'0528_y_test_{start_window}_{end_window}.npy')

X_test = np.load(x_test_path)
y_test = np.load(y_test_path)

# Insure same dataset is loaded 
assert hashlib.md5(X_test).hexdigest() == 'b7d85f42522a0a57e877422bc5947cde', "Checksum failed. Wrong file was loaded or file may be corrupted."
assert hashlib.md5(y_test).hexdigest() == '8c9cce1821372380371ade5f0ccfd4a2', "Checksum failed. Wrong file was loaded or file may be corrupted."

y_test = one_hot_encode(y_test)

print("Test Data Set:")
print("\tX Path        :", x_test_path)
print("\ty Path        :", y_test_path)
print("\tSize         :", len(X_test))
print("\tSample Shape :", X_test[0].shape)
print("\tSample Shape :", X_test.mean())
print("\tStd. Dev.    :", X_test.std())

assert len(X_test[0]) == (end_window-start_window)*2, "ERROR: Specified window does not match loaded dataset shape"

Test Data Set:
	X Path        : ../data/malab_05282024/npz/0528_X_test_0_770.npy
	y Path        : ../data/malab_05282024/npz/0528_y_test_0_770.npy
	Size         : 100000
	Sample Shape : (1540,)
	Sample Shape : 57.57549828571429
	Std. Dev.    : 845.6158899866076


## Multi-layer Model 
Or the initial "big" model 

<!-- ![Multi-layer model](../images/multi_layer_model.png) -->
<img src="../images/multi_layer_model.png" alt="alt text" width="75%">

In [5]:
"""hyperparameters"""
init_learning_rate = 1e-4
validation_split = 0
batch_size = 8192
epochs = 10
checkpoint_filename = "multi-layer.h5"
input_shape = (len(X_train_val[0]),)
start_window = 0
end_window = 770


In [6]:
sr = int((end_window-start_window)*2)
hn = sr * 2

model = Sequential()
model.add(Dense(int(hn/8), activation='relu', input_shape=(sr,)))
model.add(BatchNormalization())
model.add(Dense(2, activation='relu'))

print(model.summary())
assert model.count_params() == 595597, 'Error. Total parameters has changed.'

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 385)               593285    
                                                                 
 batch_normalization (BatchN  (None, 385)              1540      
 ormalization)                                                   
                                                                 
 dense_1 (Dense)             (None, 2)                 772       
                                                                 
Total params: 595,597
Trainable params: 594,827
Non-trainable params: 770
_________________________________________________________________
None


2024-08-20 12:35:04.743309: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory
2024-08-20 12:35:04.743359: W tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:265] failed call to cuInit: UNKNOWN ERROR (303)
2024-08-20 12:35:04.743398: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (correlator4.fnal.gov): /proc/driver/nvidia/version does not exist
2024-08-20 12:35:04.743781: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


### Train the model 

In [7]:
callbacks = [
        ModelCheckpoint(
        checkpoint_filename,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        save_freq="epoch",
    ),
    ReduceLROnPlateau(patience=75, min_delta=1**-6),
]


In [8]:
opt = Adam(learning_rate=init_learning_rate)
model.compile(
    optimizer=opt, 
    loss=CategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

history = model.fit(
    X_train_val, 
    y_train_val, 
    batch_size=batch_size,
    epochs=epochs, 
    validation_split=0.05, 
    shuffle=True, 
    callbacks=callbacks,
)

Epoch 1/10
105/105 [==============================] - 4s 34ms/step - loss: 0.2375 - accuracy: 0.9279 - val_loss: 0.1970 - val_accuracy: 0.9513 - lr: 1.0000e-04
Epoch 2/10
105/105 [==============================] - 3s 30ms/step - loss: 0.1748 - accuracy: 0.9574 - val_loss: 0.1761 - val_accuracy: 0.9554 - lr: 1.0000e-04
Epoch 3/10
105/105 [==============================] - 3s 30ms/step - loss: 0.1644 - accuracy: 0.9598 - val_loss: 0.1681 - val_accuracy: 0.9575 - lr: 1.0000e-04
Epoch 4/10
105/105 [==============================] - 3s 30ms/step - loss: 0.1577 - accuracy: 0.9610 - val_loss: 0.1632 - val_accuracy: 0.9586 - lr: 1.0000e-04
Epoch 5/10
105/105 [==============================] - 3s 30ms/step - loss: 0.1527 - accuracy: 0.9616 - val_loss: 0.1602 - val_accuracy: 0.9591 - lr: 1.0000e-04
Epoch 6/10
105/105 [==============================] - 3s 30ms/step - loss: 0.1488 - accuracy: 0.9620 - val_loss: 0.1581 - val_accuracy: 0.9594 - lr: 1.0000e-04
Epoch 7/10
105/105 [====================

### Evaluate 

In [9]:
y_pred = model.predict(X_test)
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

3125/3125 [==============================] - 8s 2ms/step
Keras  Accuracy: 0.96107


In [10]:
# get ground and excited indices 
e_indices = np.where(np.argmax(y_test, axis=1) == 1)[0]
g_indices = np.where(np.argmax(y_test, axis=1) == 0)[0]

# separate ground and excited samples 
Xe_test = X_test[e_indices]
ye_test = np.argmax(y_test, axis=1)[e_indices]

Xg_test = X_test[g_indices]
yg_test = np.argmax(y_test, axis=1)[g_indices]


In [11]:
# compute total correct for excited state 
ye_pred = model.predict(Xe_test)
e_accuracy = accuracy_score(ye_test, np.argmax(ye_pred, axis=1))

total_correct = (ye_test==np.argmax(ye_pred, axis=1)).astype(np.int8).sum()
total_incorrect = (ye_test!=np.argmax(ye_pred, axis=1)).astype(np.int8).sum()

print("Total correct:", total_correct)
print("Total incorrect:", total_incorrect)
print("Total samples:", len(Xe_test) )
print("Keras Excited Accuracy: {}".format(e_accuracy))

# compute total correct for ground state 
yg_pred = model.predict(Xg_test)
g_accuracy = accuracy_score(yg_test, np.argmax(yg_pred, axis=1))

total_correct = (yg_test==np.argmax(yg_pred, axis=1)).astype(np.int8).sum()
total_incorrect = (yg_test!=np.argmax(yg_pred, axis=1)).astype(np.int8).sum()

print("Total correct:", total_correct)
print("Total incorrect:", total_incorrect)
print("Total samples:", len(Xg_test) )
print("Keras Ground Accuracy: {}".format(g_accuracy))

# compute fidelity 
fidelity = 0.5*(e_accuracy + g_accuracy)
print('\n===================================')
print('Fidelity', fidelity)
print('===================================')

1563/1563 [==============================] - 4s 2ms/step
Total correct: 47384
Total incorrect: 2616
Total samples: 50000
Keras Excited Accuracy: 0.94768
1563/1563 [==============================] - 4s 3ms/step
Total correct: 48723
Total incorrect: 1277
Total samples: 50000
Keras Ground Accuracy: 0.97446

Fidelity 0.96107


## Single-layer Model 
Or the "small" model 

<img src="../images/single_layer_model.png" alt="alt text" width="50%">

In [12]:
"""hyperparameters"""
init_learning_rate = 1e-4
validation_split = 0
batch_size = 8192
epochs = 10
checkpoint_filename = "single-layer.h5"
input_shape = (len(X_train_val[0]),)
start_window = 0
end_window = 770


In [13]:
model = Sequential()
model.add(Dense(2, activation='relu', input_shape=(sr,)))
model.add(BatchNormalization())

print(model.summary())
assert model.count_params() == 3090, 'Error. Total parameters has changed.'

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_2 (Dense)             (None, 2)                 3082      
                                                                 
 batch_normalization_1 (Batc  (None, 2)                8         
 hNormalization)                                                 
                                                                 
Total params: 3,090
Trainable params: 3,086
Non-trainable params: 4
_________________________________________________________________
None


### Train the model 

In [14]:
callbacks = [
        ModelCheckpoint(
        checkpoint_filename,
        monitor="val_loss",
        verbose=0,
        save_best_only=True,
        save_weights_only=False,
        save_freq="epoch",
    ),
    ReduceLROnPlateau(patience=75, min_delta=1**-6),
]


In [15]:
opt = Adam(learning_rate=init_learning_rate)
model.compile(
    optimizer=opt, 
    loss=CategoricalCrossentropy(from_logits=True), 
    metrics=['accuracy']
)

history = model.fit(
    X_train_val, 
    y_train_val, 
    batch_size=batch_size,
    epochs=epochs, 
    validation_split=0.05, 
    shuffle=True, 
    callbacks=callbacks,
)

Epoch 1/10
105/105 [==============================] - 2s 16ms/step - loss: 0.6008 - accuracy: 0.6870 - val_loss: 0.3975 - val_accuracy: 0.8354 - lr: 1.0000e-04
Epoch 2/10
105/105 [==============================] - 1s 14ms/step - loss: 0.3645 - accuracy: 0.8877 - val_loss: 0.2941 - val_accuracy: 0.9169 - lr: 1.0000e-04
Epoch 3/10
105/105 [==============================] - 1s 14ms/step - loss: 0.3008 - accuracy: 0.9323 - val_loss: 0.2626 - val_accuracy: 0.9387 - lr: 1.0000e-04
Epoch 4/10
105/105 [==============================] - 1s 14ms/step - loss: 0.2718 - accuracy: 0.9443 - val_loss: 0.2477 - val_accuracy: 0.9458 - lr: 1.0000e-04
Epoch 5/10
105/105 [==============================] - 1s 14ms/step - loss: 0.2552 - accuracy: 0.9498 - val_loss: 0.2386 - val_accuracy: 0.9504 - lr: 1.0000e-04
Epoch 6/10
105/105 [==============================] - 1s 14ms/step - loss: 0.2443 - accuracy: 0.9529 - val_loss: 0.2320 - val_accuracy: 0.9520 - lr: 1.0000e-04
Epoch 7/10
105/105 [====================

### Evaluate 

In [16]:
y_pred = model.predict(X_test)
print("Keras  Accuracy: {}".format(accuracy_score(np.argmax(y_test, axis=1), np.argmax(y_pred, axis=1))))

3125/3125 [==============================] - 5s 2ms/step
Keras  Accuracy: 0.95684


In [17]:
# get ground and excited indices 
e_indices = np.where(np.argmax(y_test, axis=1) == 1)[0]
g_indices = np.where(np.argmax(y_test, axis=1) == 0)[0]

# separate ground and excited samples 
Xe_test = X_test[e_indices]
ye_test = np.argmax(y_test, axis=1)[e_indices]

Xg_test = X_test[g_indices]
yg_test = np.argmax(y_test, axis=1)[g_indices]


In [18]:
# compute total correct for excited state 
ye_pred = model.predict(Xe_test)
e_accuracy = accuracy_score(ye_test, np.argmax(ye_pred, axis=1))

total_correct = (ye_test==np.argmax(ye_pred, axis=1)).astype(np.int8).sum()
total_incorrect = (ye_test!=np.argmax(ye_pred, axis=1)).astype(np.int8).sum()

print("Total correct:", total_correct)
print("Total incorrect:", total_incorrect)
print("Total samples:", len(Xe_test) )
print("Keras Excited Accuracy: {}".format(e_accuracy))

# compute total correct for ground state 
yg_pred = model.predict(Xg_test)
g_accuracy = accuracy_score(yg_test, np.argmax(yg_pred, axis=1))

total_correct = (yg_test==np.argmax(yg_pred, axis=1)).astype(np.int8).sum()
total_incorrect = (yg_test!=np.argmax(yg_pred, axis=1)).astype(np.int8).sum()

print("Total correct:", total_correct)
print("Total incorrect:", total_incorrect)
print("Total samples:", len(Xg_test) )
print("Keras Ground Accuracy: {}".format(g_accuracy))

# compute fidelity 
fidelity = 0.5*(e_accuracy + g_accuracy)
print('\n===================================')
print('Fidelity', fidelity)
print('===================================')

1563/1563 [==============================] - 3s 2ms/step
Total correct: 46892
Total incorrect: 3108
Total samples: 50000
Keras Excited Accuracy: 0.93784
1563/1563 [==============================] - 2s 1ms/step
Total correct: 48792
Total incorrect: 1208
Total samples: 50000
Keras Ground Accuracy: 0.97584

Fidelity 0.95684
